In [37]:
import types
from library.localization import *
from deconv_utils import *
from library.image_utils import *
import csv
import sys

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

#Validation & Model File Inputs
data_file = "Data/validation_1"
data_extn = ".pkl"
results_file = data_file + "_results.csv"
model_file = 'Data/vgg16_weights.h5'
class_file = 'Data/CLASSES.pkl'

#Image Processing Inputs
Param.num_dilation = 15

start_index = 0
try:
    #Check if csv file exists and read the last index outputted
    results_csv = open(results_file,'r')
    lastline = results_csv.read().split('\n')[-2]
    start_index = int(lastline.split(',')[0]) + 1
except IOError as e:
    pass

#Write to CSV file
results_csv = open(results_file,'a')

#Load the VGG Model and ImageNet Class Mappings
from library.classifiers.pretrained_vgg16 import PretrainedVGG
model = PretrainedVGG(h5_file = model_file)
print "Loaded Model"
CLASSES = pickle.load(open(class_file))
print "Loaded Class File"

#Read Input File for the candidates
candidates = pickle.load(open(data_file+data_extn))
print "Loaded Input Data Set"

#Read Input File and start at the index higher than that of last line

for index in range(start_index,len(candidates)):
    ret = candidates[index]
    class_wnid,imgid,class_idx,xmlidx,url,xmin,xmax,ymin,ymax = ret
    im = image_from_url(url)
    if type(im) == types.NoneType:
        #Debug Print
        print "[%d] Skip (%d,%d): %s : URL is bad."%(index, class_idx,xmlidx, url)
    elif np.mean(im)>=253:
        #Debug Print
        print "[%d] Skip (%d,%d): %s : URL is Empty(White) Image"%(index, class_idx,xmlidx,url)
    else:  
        bbox_coords = bbox(im,model,class_no=class_idx,n_neurons = 5,kmax=10)
        xmin_out, xmax_out, ymin_out, ymax_out = bbox_coords
        precision = eval_precision(bbox_coords, (xmin,xmax,ymin,ymax))
        #Write Results to file
        result_row = "%d, %s, %d, %d, %f,"%(index, imgid, class_idx, xmlidx, precision)
        result_row += "%d, %d, %d, %d,"%(xmin, xmax, ymin, ymax,)
        result_row += "%d, %d, %d, %d"%(xmin_out, xmax_out, ymin_out, ymax_out)
        result_row += "\n"
        results_csv.write(result_row)
        results_csv.flush()
        #Debug Print
        print "[%d] %d,%d,%f"%(index, class_idx,xmlidx,precision)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded Model
Loaded Class File
Loaded Input Data Set
[8] 11,349,0.414949
[9] Skip (208,65): http://farm2.static.flickr.com/1246/905331985_1f0065e586.jpg : URL is Empty(White) Image
[10] 338,246,0.404768


KeyboardInterrupt: 